<a href="https://colab.research.google.com/github/sp1667/Suryaprakashchary_INFO5502_SPRING2022/blob/main/lab_assignment_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The third Lab-assignment (02/10/2022, 50 points in total)

The purpose of this exercise is to understand users' information needs, then collect data from different sources for analysis.

Question 1 (10 points). Fomulate your domain problem: Describe an interesting research question (or practical question) you have in mind, what kind of data should be collected to answer the question(s)? How many data needed for the analysis? The detail steps for collecting and save the data. 

In [ ]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''
Please write you answer here:

Domain problem: 
. My domain problem is to analyse the text data that obtained from the interviews of histories of people.

Research question:
. Do the Topic Modelling of the topics that narrators talked about during the interview.
. Do EDA on the Location, Year of Birth, Gender, Generation data of the narrators.

Data Required to answer the questions:
The following are the attributes of the required data to answer the research question
. Bio of the narrator
. Transcript details such as interview questions and answers


We can gather 1042 interview samples from the following link: https://ddr.densho.org/narrators/


Steps for collecting and saving the data:
. We use web scraping technique to scrape the data.
. We use BeautifulSoup library to parse the Densho Digital Repository.
. There are 37 pages of narrators.
. We parse through each page and go to the Narrator title link and then to interviews hyperlink.
. We get the bio from the Narrator homepage.
. The interview segment details are obtained from the transcript hyperlink.
. We parse the Segment Body-> Paragraph elements in the html code and then store the text data in a json format.
. The json format is the prepared data for our research.


'''

Question 2 (10 points). Collect your data to answer the research problem: Write python code to collect 1000 data samples you discussed above.

In [ ]:
# You code here (Please add comments in the code):
# The below is for collectig the 1042 samples from 904 narrators from the https://ddr.densho.org/narrators
# We collect the data and save it in JSON format.

# importing the libraries...
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.error import HTTPError
import json
import re

total_count = 0
count = 0
interview_count = 0
# url of the densho digital repository to scrape the data
main_url = "https://ddr.densho.org/narrators/?page={}"
for page_num in range(1, 38):
    # we create the data soup of the main url page html elements.
    link1 = Request(main_url.format(page_num), headers={'User-Agent': 'Mozilla/5.0'})
    url1 = urlopen(link1)

    data1 = url1.read()
    data1_soup = BeautifulSoup(data1)
    
    #print("*** Page {} ***".format(page_num))
    # we iterate through each narrator page link
    for narrator_link in data1_soup.find_all('h4'):
        link2 = Request(narrator_link.a.get('href'), headers={'User-Agent': 'Mozilla/5.0'})
        url2 = urlopen(link2)

        data2 = url2.read()
        data2_soup = BeautifulSoup(data2)
        count = count + 1
        
        # *** Interview links page***
        # we find the div and attributes of the div and fetch the required values
        narrator = data2_soup.find_all("div", attrs={'class':'col-sm-8 col-md-8'})[0]
        if data2_soup.find_all("div", attrs={'class':'url'}) != []:
            
            # Some narrators have given than one interview
            # we iterate through each interview of the narrator
            for interviews in data2_soup.find_all("div", attrs={'class':'url'}):
                result = {}
                result['Narrator_Name'] = ""
                result['Bio'] = ""
                result['Title'] = ""
                result['Interviewer'] = ""
                result['Location'] = ""
                result['Date'] = ""
                result['Densho ID'] = ""
                result['transcript'] = {}
                # Narrator name and Bio...
                try:
                    result['Narrator_Name'] = narrator.h1.text.strip().replace('"', "")#re.sub('"', '\"', narrator.h1.text.strip())
                except:
                    result['Narrator_Name'] = ""
                    
                try:
                    result['Bio'] = narrator.p.text
                except :
                    result['Bio'] = ""
                
                # we go to interview page and fetch details from it
                link3 = Request(interviews.a.get('href'), headers={'User-Agent': 'Mozilla/5.0'})
                url3 = urlopen(link3)

                data3 = url3.read()
                data3_soup = BeautifulSoup(data3)
                #print("*** Segment Descriptions ***")
                num = 1
                # we go to the 'tbody' element and find all 'td' elements
                for desc in data3_soup.find_all('tbody')[0].find_all('td'):
                    result['transcript']['segment {}'.format(num)] = {}
                    result['transcript']['segment {}'.format(num)]['Segment Desc'] = desc.text.split(' (', 1)[0].strip()
                    result['transcript']['segment {}'.format(num)]['Segment Q&A'] = {}
                    num = num + 1
                
                for transcript_link in data3_soup.find_all('div', attrs={'id':'dls_tab'}):
                    #print("*** Transcript link ***")
                    try:
                        #print(transcript_link.find_all('a')[1].get('href'))
                        link4 = Request(transcript_link.find_all('a')[1].get('href'), headers={'User-Agent': 'Mozilla/5.0'})
                        url4 = urlopen(link4)

                        data4 = url4.read()
                        data4_soup = BeautifulSoup(data4)

                        #print("-----------------------------------------")
                        for data in data4_soup.find_all('div', attrs={'class':'segmentHead'}):
                            # title, interviewer, location, date, densho id...
                            for attr in data.text.splitlines():
                                if 'Title' in attr:
                                    #print(i.strip())
                                    result['Title'] = ' '.join(attr.strip().split()[1:])
                                if 'Interviewer' in attr:
                                    #print(i.strip())
                                    result['Interviewer'] = ' '.join(attr.strip().split()[1:])
                                if 'Location' in attr:
                                    #print(i.strip())
                                    result['Location'] = ' '.join(attr.strip().split()[1:])
                                if 'Date' in attr:
                                    #print(i.strip())
                                    result['Date'] = ' '.join(attr.strip().split()[1:])
                                if 'Densho ID' in attr:
                                    #print(i.strip())
                                    result['Densho ID'] = ' '.join(attr.strip().split()[2:])
                        
                        num = 1
                        # An interview has multiple segments
                        # Each segment has questions and answer
                        for seg in data4_soup.find_all('div', attrs={'class':'segmentBody'}):
                            # Each segment will have multiple paragraph elements
                            for p in range(0, len(seg.find_all('p')[1:-1])):
                                if '?' in seg.find_all('p')[p].text:
                                    question = re.sub('\s+',' ', re.sub("  +", "",seg.find_all('p')[p].text))
                                    question = " ".join(i for i in question.split()[1:])
                                    try:
                                        # we remove unnecessary white spaces...
                                        answer = re.sub('\s+',' ', re.sub("  +", "",seg.find_all('p')[p+1].text))
                                        answer = " ".join(i for i in answer.split()[1:])
                                    except:
                                        answer = ""
                                    result['transcript']['segment {}'.format(num)]['Segment Q&A'][question] = answer
                            num = num + 1
                                                
                    #except HTTPError as err:
                    except Exception as e:
                        #print("No Transcripts")
                        #print(e)
                        #print(len(data3_soup.find_all('tbody')[0].find_all('td')))
                        result['Title'] = ""
                        result['Interviewer'] = ""
                        result['Location'] = ""
                        result['Date'] = ""
                        result['Densho ID'] = ""
                        #num = 1
                        result['transcript'] = {}
                        for num in range(0, len(data3_soup.find_all('tbody')[0].find_all('td'))):
                            result['transcript']['segment {}'.format(num+1)] = {}
                            result['transcript']['segment {}'.format(num+1)]['Segment Desc'] = ""
                            result['transcript']['segment {}'.format(num+1)]['Segment Q&A'] = {}
                            #num = num + 1
                '''
                import os.path
                if os.path.isfile(r'{}.json'.format(result['Narrator_Name'] + '-' + str(interview_count))):
                    with open(r'{}.json'.format(result['Narrator_Name'] + '(1)' + '-' + str(interview_count)), 'w', encoding='utf-8') as f:
                        json.dump(result, f, ensure_ascii=False, indent=4)
                else:
                    with open(r'{}.json'.format(result['Narrator_Name'] + '-' + str(interview_count)), 'w', encoding='utf-8') as f:
                        json.dump(result, f, ensure_ascii=False, indent=4)
                print('***************************')
                '''
                interview_count = interview_count + 1 
             
        else:
            # some narrators have no interviews
            # In that case, we fill the json file with empty values.
            '''
            print("%%%%%%%%%%%%%%%%%%%%%%%%%")
            print("No Interviews....")
            print("%%%%%%%%%%%%%%%%%%%%%%%%%")
            '''
            result= {}
            result['Narrator_Name'] = narrator.h1.text.strip().replace('"', '')#re.sub('"', '\"', narrator.h1.text.strip())
            result['Bio'] = narrator.p.text
            result['Title'] = ""
            result['Interviewer'] = ""
            result['Location'] = ""
            result['Date'] = ""
            result['Densho ID'] = ""
            result['transcript'] = {}
            interview_count = interview_count + 1
            #print(result)
            '''
            with open(r'{}.json'.format(result['Narrator_Name']), 'w', encoding='utf-8') as f:
                json.dump(result, f, ensure_ascii=False, indent=4)
                        
            print("****************************")    
            '''
    
print("Total count of narrators is: {}".format(count))
print("Total count of Samples is: {}".format(interview_count))

Question 3 (10 points). Understand the data quality: Search a second hand dataset (any dataset) from kaggle or other websites. Describe the data quality problem of the dataset and explain your strtegy to clean the data.

In [ ]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):
'''

Please write you answer here:

-> I considered a text classification data from Kaggle.
Dataset:
. Link: https://www.kaggle.com/team-ai/spam-text-message-classification
. There are 2 data columns in the dataset.
. The target column is having 2 classes Spam or Not Spam.

Steps to Clean the Text classification data:
. we convert all the upper case to lower case character if any.
. Remove all the stopwords from the data.
. Remove special characters and unwanted spaces from the data.
. Perform stemming and Lemmatization on the data.
. Check if there is any imbalance in the data.
. If there is any imbalance, we use SMOTE technique to balance the data by performing over sampling and under sampling.
. Then, we can build a classifier model to classify the data as spam or not spam.
'''

Question 4 (20 points). Data cleaning: There are two datasets TwADR-L (from Twitter) and AskAPatient (Link: https://zenodo.org/record/55013#.YgU2NN-ZO4T) for medical concept
normalization. However, the two datasets have serious data quality problems. Please read the introduction of the datasets and clean the two datasets by following the steps in the statement.

In the original dataset, the TwADR-L had 48,057 training, 1,256 validation and 1,427 test examples. The test set (all
test samples from 10 folds combined) consists of 765 unique phrases and 273 unique classes (medical concepts). The AskAPatient dataset contained 156,652 training, 7,926 validation, and 8,662 test examples. The entire test set (all test samples
from 10 folds combined) consists of 3,749 unique phrases and 1,035 unique classes (medical concepts). The authors
randomly split each dataset into ten equal folds, ran 10-fold cross validation and reported the accuracy averaged across the
ten folds. 

We found that, in the original data set, many phrase-label pairs appeared multiple times within the same training data file
and also across the training and test data sets in the same fold. In the AskAPatient data set, on average 35.82% of the test data overlapped with training data in the same fold. In the Twitter (TwADR-L) dataset, on average 8.62% of the test set had an overlap with the training data in the same fold. Having a large overlap between the training and the test data can potentially
introduce bias in the model and contribute to high accuracy. It is not unlikely that the high model performance reported in the original paper may be triggered by the the large overlap between the training and test sets.

Therefore to remove the bias, we further cleaned and recreated the training, validation, and test sets such that each
phrase-label pair appears only once in the entire dataset (either in training, validation or test set).

(1) First, we combined all examples in training, validation and test data from the original data set and then removed all
duplicate phrase-label pairs (examples that have the same phrase and label pair and appear more than once in training/validation/test datasets). Table II shows the statistics of the new dataset (after removing duplicates). The Twitter data set had 3,157 unique phrase-label pairs and 2,220 unique labels (medical concepts) while 173 phrases had multiple labels (i.e., they were assigned to more than one label). Many concepts had only one example, and the concept that had the most number
of examples had 36 phrases. On average, each concept had 1.42 examples. The AskAPatient data set had 4,496 unique phrase-label pairs, 1,036 unique labels while 26 phrases had multiple labels. Table III shows examples of phrases that had multiple labels. For example, ‘mad’ can be mapped to ‘anger’ or ‘rage’ and ‘sore’ can be mapped to ‘pain’ or ‘myalgia’.

(2) Second, we remove all concepts that had less than five examples. The statistics of the final data are shown in Table IV.

(3) Third, we divide all examples without multiple labels into random 10 folds such that each unique phrase-label pair
appears once in one of the 10 test sets. We add the pairs with multiple labels into the training data. This final 10-folds
dataset is used in all our experiments.

(The original paper can be download on canvas)

In [ ]:
# 1.)

import glob
result_df =  pd.DataFrame()
# we iterate through both datasets 
for dataset in ['TwADR-L', 'AskAPatient']:
    result = {}
    df = pd.DataFrame()
    for filepath in glob.glob('/content/gdrive/My Drive/datasets/{}/*.txt'.format(dataset)):
        data = pd.read_csv(filepath, sep = "\t", header = None, encoding= 'unicode_escape')
        df = df.append(data)
    # we create combined dataframe by combining Each dataset text files.
    df = df.reset_index(drop=True)
    df['phrase_label'] = df[1] + " " + df[2]
    df.columns = ['id', 'labels', 'phrases', 'phrase-label']
    # type casting to str
    df = df.astype({"id": str})
    # converting all values to lower case
    for columns in df.columns:
        df[columns] = df[columns].str.lower() 

    # dropping duplicates
    df = df.drop_duplicates('phrase-label')

    # storing results of the table in dictionary
    result['Unique_phrases'] = len(df['phrases'].unique())
    result['Unique_labels'] = len(df['labels'].unique())
    result['Unique_phrase_label_pairs'] = df.shape[0]
    df1 = pd.DataFrame(df['phrases'].value_counts())
    result['Phrases with multiple labels'] = df1[df1['phrases'] > 1].shape[0]
    result['Min examples per label'] = df['labels'].value_counts().values.min()
    result['Max examples per label'] = df['labels'].value_counts().values.max()
    result['Avg examples per label'] = round(df['labels'].value_counts().mean(), 2)
    result_df = result_df.append(result, ignore_index=True)


result_df = result_df.T
result_df = result_df.astype({0: int, 1: int})
result_df.columns = ['TwADR-L', 'AskAPatient']

result_df

,TwADR-L,AskAPatient
Unique_phrases,2944,4470
Unique_labels,2220,1038
Unique_phrase_label_pairs,3157,4507
Phrases with multiple labels,173,35
Min examples per label,1,1
Max examples per label,36,141
Avg examples per label,1,4


In [ ]:
# 2.)

import glob
result_df = pd.DataFrame()
# we iterate through both datasets 
for dataset in ['TwADR-L', 'AskAPatient']:
    df = pd.DataFrame()
    result = {}
    for filepath in glob.glob('/content/gdrive/My Drive/datasets/{}/*.txt'.format(dataset)):
        #print(filepath)
        data = pd.read_csv(filepath, sep = "\t", header = None, encoding= 'unicode_escape')
        df = df.append(data)
    # we create combined dataframe by combining Each dataset text files.
    df = df.reset_index(drop=True)
    df['phrase_label'] = df[1] + " " + df[2]
    df.columns = ['id', 'labels', 'phrases', 'phrase-label']
    #print(df.shape)
    df = df.astype({"id": str})
    # converting to lower case
    for columns in df.columns:
        df[columns] = df[columns].str.lower() 

    # we drop duplicates that are phrase-label pairs... 
    df = df.drop_duplicates('phrase-label')

    index_list = []
    for i in range(df.shape[0]):
        if df['labels'].value_counts()[df.iloc[i]['labels']] < 5:
            index_list.append(i)

    # we drop the labels that have less than count of 5.
    df.drop(df.index[index_list], inplace=True)
    result = {}
    # storing results of the table in dictionary
    result['Unique_phrases'] = len(df['phrases'].unique())
    result['Unique_labels'] = len(df['labels'].unique())
    result['Unique_phrase_label_pairs'] = df.shape[0]
    df1 = pd.DataFrame(df['phrases'].value_counts())
    result['Phrases with multiple labels'] = df1[df1['phrases'] > 1].shape[0]
    result['Min examples per label'] = df['labels'].value_counts().values.min()
    result['Max examples per label'] = df['labels'].value_counts().values.max()
    result['Avg examples per label'] = round(df['labels'].value_counts().mean(), 2)
    # appending dictinary to dataframe
    result_df = result_df.append(result, ignore_index=True)

result_df = result_df.T
result_df = result_df.astype({0: int, 1: int})
result_df.columns = ['TwADR-L', 'AskAPatient']

result_df

,TwADR-L,AskAPatient
Unique_phrases,616,2665
Unique_labels,76,233
Unique_phrase_label_pairs,721,2686
Phrases with multiple labels,87,19
Min examples per label,5,5
Max examples per label,36,141
Avg examples per label,9,11
